We create and start an IPython parallel cluster

In [1]:
import ipyparallel as ipp
cluster = ipp.Cluster(n=4, engines="mpi")
rc = await cluster.start_and_connect(activate=True)

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

What does it mean starting a cluster?

In [2]:
cluster.engine_set.args

['mpiexec',
 '-n',
 '4',
 '/home/dokken/miniconda3/envs/mpi-tutorial/bin/python',
 '-m',
 'ipyparallel.engine',
 '--mpi']

This means that we have `4` asynchronous processes, which means that we can return to working on a single process prior to the other processes finishing. For instance, we can make the following simple code

In [36]:
def hello(wait_time: float):
    from mpi4py import MPI
    import os
    import time
    rank = MPI.COMM_WORLD.rank
    pid = os.getpid()
    print(f"Hello I am process {rank} with Unix ID {pid}")
    if rank == 0:
        time.sleep(wait_time)
    return (rank, pid)

We next execute this on the different processes.

In [37]:
import time
process_view = rc[:]
wait_time = 1
start = time.perf_counter()
result = process_view.apply_async(hello, wait_time)
sent = time.perf_counter()
print(f"Result computed for all procs: {result.ready()}")
result.wait()
end = time.perf_counter()
print(f"Time to send function to processes {sent - start:.2e}s")
print(f"Time for all to finish {end-sent:.2f}s")

Result computed for all procs: False
Time to send function to processes 2.80e-03s
Time for all to finish 1.01s


We can also do this in a synchronous way

In [38]:
start_sync = time.perf_counter()
sync_result = process_view.apply_sync(hello, wait_time)
end_sync = time.perf_counter()
print(f"Time for all to finish {end_sync-start_sync}")
assert result.get() == sync_result
print(sync_result)

Time for all to finish 1.0128639069998826
[(0, 32742), (1, 32743), (2, 32744), (3, 32745)]


We could also work on a subset of the processes we have started

In [40]:
p2and3 = rc[2, 3]
start = time.perf_counter()
result2and3 = p2and3.apply_async(hello, wait_time)
sent = time.perf_counter()
print(f"Result computed for all procs: {result2and3.ready()}")
result2and3.wait()
end = time.perf_counter()
print(
    f"Runtime for subset to finish operation {end-start} {result2and3.get()}")

Result computed for all procs: False
Runtime for subset to finish operation 0.018454205000125512 [(2, 32744), (3, 32745)]
